# Train RNN

Here we give a complete example of what using Equinox normally looks like day-to-day.

In this example we'll train an RNN to classify clockwise vs anticlockwise spirals.

In [1]:
import math

import jax
import jax.lax as lax
import jax.numpy as jnp
import jax.random as jrandom
import optax  # https://github.com/deepmind/optax

import equinox as eqx

We begin by importing the usual libraries, setting up a very simple dataloader, and generating a toy dataset of spirals.

In [2]:
def dataloader(arrays, batch_size, *, key):
    dataset_size = arrays[0].shape[0]
    assert all(array.shape[0] == dataset_size for array in arrays)
    indices = jnp.arange(dataset_size)
    while True:
        perm = jrandom.permutation(key, indices)
        (key,) = jrandom.split(key, 1)
        start = 0
        end = batch_size
        while end < dataset_size:
            batch_perm = perm[start:end]
            yield tuple(array[batch_perm] for array in arrays)
            start = end
            end = start + batch_size


def get_data(dataset_size, *, key):
    t = jnp.linspace(0, 2 * math.pi, 16)
    offset = jrandom.uniform(key, (dataset_size, 1), minval=0, maxval=2 * math.pi)
    x1 = jnp.sin(t + offset) / (1 + t)
    x2 = jnp.cos(t + offset) / (1 + t)
    y = jnp.ones((dataset_size, 1))

    half_dataset_size = dataset_size // 2
    x1 = x1.at[:half_dataset_size].multiply(-1)
    y = y.at[:half_dataset_size].set(0)
    x = jnp.stack([x1, x2], axis=-1)

    return x, y

Now for our model.

Purely by way of example, we handle the final adding on of bias ourselves, rather than letting the `linear` layer do it. This is just so we can demonstrate how to use custom parameters in models.

In [3]:
class RNN(eqx.Module):
    hidden_size: int
    cell: eqx.Module
    linear: eqx.nn.Linear
    bias: jnp.ndarray

    def __init__(self, in_size, out_size, hidden_size, *, key):
        ckey, lkey = jrandom.split(key)
        self.hidden_size = hidden_size
        self.cell = eqx.nn.GRUCell(in_size, hidden_size, key=ckey)
        self.linear = eqx.nn.Linear(hidden_size, out_size, use_bias=False, key=lkey)
        self.bias = jnp.zeros(out_size)

    def __call__(self, input):
        hidden = jnp.zeros((self.hidden_size,))

        def f(carry, inp):
            return self.cell(inp, carry), None

        out, _ = lax.scan(f, hidden, input)
        # sigmoid because we're performing binary classification
        return jax.nn.sigmoid(self.linear(out) + self.bias)

And finally the training loop.

In [4]:
def main(
    dataset_size=10000,
    batch_size=32,
    learning_rate=3e-3,
    steps=200,
    hidden_size=16,
    depth=1,
    seed=5678,
):
    data_key, loader_key, model_key = jrandom.split(jrandom.PRNGKey(seed), 3)
    xs, ys = get_data(dataset_size, key=data_key)
    iter_data = dataloader((xs, ys), batch_size, key=loader_key)

    model = RNN(in_size=2, out_size=1, hidden_size=hidden_size, key=model_key)

    @eqx.filter_jit
    @eqx.filter_value_and_grad
    def make_step(model, x, y):
        pred_y = jax.vmap(model)(x)
        # Trains with respect to binary cross-entropy
        return -jnp.mean(y * jnp.log(pred_y) + (1 - y) * jnp.log(1 - pred_y))

    optim = optax.adam(learning_rate)
    opt_state = optim.init(model)
    for step, (x, y) in zip(range(steps), iter_data):
        loss, grads = make_step(model, x, y)
        loss = loss.item()
        updates, opt_state = optim.update(grads, opt_state)
        model = eqx.apply_updates(model, updates)
        print(f"{step=}, {loss=}")

    pred_ys = jax.vmap(model)(xs)
    num_correct = jnp.sum((pred_ys > 0.5) == ys)
    final_accuracy = (num_correct / dataset_size).item()
    print(f"{final_accuracy=}")

`eqx.filter_jit` will look at all the arguments passed to `make_step`, and automatically JIT-trace every array and JIT-static everything else. For example the `model` parameters, and the data `x` and `y` will be traced. (However `model.hidden_size` is an integer, so it will be static'd instead.) This is a convenient way to JIT when your arguments are PyTrees, and if you need finer control then `eqx.filter_jit` takes a `filter_spec` argument.

`eqx.filter_value_and_grad` will calculate the gradient with respect to the first argument (`model`). It will automatically do so for just the floating-point arrays in the `model` PyTree (i.e. its parameters) but not for anything else (e.g. `model.hidden_size`). Once again if you need finer control then `eqx.filter_value_and_grad` takes a `filter_spec` argument.

_(Note that the above code is written for clarity, not speed. If you want maximum performance then the operations inside `dataloader`, and the `optim.update`, `eqx.apply_updates` operations, should all be brought inside the single JIT operation.)_

In [5]:
main()  # All right, let's run the code.

step=0, loss=0.695469856262207
step=1, loss=0.6940110921859741
step=2, loss=0.6931639909744263
step=3, loss=0.6961429119110107
step=4, loss=0.6957006454467773
step=5, loss=0.6920372247695923
step=6, loss=0.6938458681106567
step=7, loss=0.6933101415634155
step=8, loss=0.6897573471069336
step=9, loss=0.6940688490867615
step=10, loss=0.6896819472312927
step=11, loss=0.6991991996765137
step=12, loss=0.691841185092926
step=13, loss=0.6914103627204895
step=14, loss=0.7029447555541992
step=15, loss=0.7040634155273438
step=16, loss=0.6862525939941406
step=17, loss=0.700591504573822
step=18, loss=0.6833503246307373
step=19, loss=0.6996323466300964
step=20, loss=0.6967240571975708
step=21, loss=0.6946160793304443
step=22, loss=0.6940494775772095
step=23, loss=0.688583493232727
step=24, loss=0.694453239440918
step=25, loss=0.694945216178894
step=26, loss=0.6929985284805298
step=27, loss=0.6932469606399536
step=28, loss=0.6898558735847473
step=29, loss=0.6925642490386963
step=30, loss=0.6929633617